In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier



The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [5]:
import zipfile

with zipfile.ZipFile("parkinson+s+disease+classification.zip", "r") as zip_ref:
    zip_ref.extractall("parkinsons_data")

In [6]:
import os
os.listdir("parkinsons_data")

['pd_speech_features.rar']

In [7]:
!apt-get install unrar
!unrar x parkinsons_data/pd_speech_features.rar parkinsons_data/

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.

UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from parkinsons_data/pd_speech_features.rar

Extracting  parkinsons_data/pd_speech_features.csv                         1%  3%  4%  6%  7%  9% 10% 12% 13% 15% 16% 18% 19% 21% 22% 24% 26% 27% 29% 30% 32% 33% 35% 36% 38% 39% 41% 42% 44% 45% 47% 48% 50% 52% 53% 55% 56% 58% 59% 61% 62% 64% 65% 67% 68% 70% 71% 73% 74% 76% 78% 79% 81% 82% 84% 85% 87% 88% 90% 91% 93% 94% 96% 97% 99%100%  OK 
All OK


In [8]:
import os
os.listdir("parkinsons_data")

['pd_speech_features.rar', 'pd_speech_features.csv']

In [9]:
import pandas as pd

df = pd.read_csv("parkinsons_data/pd_speech_features.csv", header=[1])

df

,id,gender,PPE,DFA,RPDE,numPulses,numPeriodsPulses,meanPeriodPulses,stdDevPeriodPulses,locPctJitter,...,tqwt_kurtosisValue_dec_28,tqwt_kurtosisValue_dec_29,tqwt_kurtosisValue_dec_30,tqwt_kurtosisValue_dec_31,tqwt_kurtosisValue_dec_32,tqwt_kurtosisValue_dec_33,tqwt_kurtosisValue_dec_34,tqwt_kurtosisValue_dec_35,tqwt_kurtosisValue_dec_36,class
0,0,1,0.85247,0.71826,0.57227,240,239,0.008064,0.000087,0.00218,...,1.5620,2.6445,3.8686,4.2105,5.1221,4.4625,2.6202,3.0004,18.9405,1
1,0,1,0.76686,0.69481,0.53966,234,233,0.008258,0.000073,0.00195,...,1.5589,3.6107,23.5155,14.1962,11.0261,9.5082,6.5245,6.3431,45.1780,1
2,0,1,0.85083,0.67604,0.58982,232,231,0.008340,0.000060,0.00176,...,1.5643,2.3308,9.4959,10.7458,11.0177,4.8066,2.9199,3.1495,4.7666,1
3,1,0,0.41121,0.79672,0.59257,178,177,0.010858,0.000183,0.00419,...,3.7805,3.5664,5.2558,14.0403,4.2235,4.6857,4.8460,6.2650,4.0603,1
4,1,0,0.32790,0.79782,0.53028,236,235,0.008162,0.002669,0.00535,...,6.1727,5.8416,6.0805,5.7621,7.7817,11.6891,8.2103,5.0559,6.1164,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,250,0,0.80903,0.56355,0.28385,417,416,0.004627,0.000052,0.00064,...,3.0706,3.0190,3.1212,2.4921,3.5844,3.5400,3.3805,3.2003,6.8671,0
752,250,0,0.16084,0.56499,0.59194,415,413,0.004550,0.000220,0.00143,...,1.9704,1.7451,1.8277,2.4976,5.2981,4.2616,6.3042,10.9058,28.4170,0
753,251,0,0.88389,0.72335,0.46815,381,380,0.005069,0.000103,0.00076,...,51.5607,44.4641,26.1586,6.3076,2.8601,2.5361,3.5377,3.3545,5.0424,0
754,251,0,0.83782,0.74890,0.49823,340,339,0.005679,0.000055,0.00092,...,19.1607,12.8312,8.9434,2.2044,1.9496,1.9664,2.6801,2.8332,3.7131,0


**Run 4 Models with ALl FEATURES**


In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score

# ML models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

# =====================
# 1. Load dataset
# =====================

# Features and target
X = df.drop(columns=["class"]).values
y = df["class"].values

# Encode target
le = LabelEncoder()
y = le.fit_transform(y)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Standardize features (important for SVM, Logistic, NN)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# =====================
# 2. Logistic Regression
# =====================
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)
acc_log = accuracy_score(y_test, y_pred)

# =====================
# 3. Random Forest
# =====================
rf = RandomForestClassifier(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
acc_rf = accuracy_score(y_test, y_pred)

# =====================
# 4. Support Vector Machine
# =====================
svm = SVC(kernel='rbf', probability=True, random_state=42)
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)
acc_svm = accuracy_score(y_test, y_pred)

# =====================
# 5. XGBoost
# =====================
xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
acc_xgb = accuracy_score(y_test, y_pred)

# =====================
# 6. Neural Network (Deep Learning)
# =====================
# One-hot encode target
y_train_nn = to_categorical(y_train)
y_test_nn = to_categorical(y_test)

nn = Sequential()
nn.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
nn.add(Dropout(0.3))
nn.add(Dense(32, activation='relu'))
nn.add(Dense(y_train_nn.shape[1], activation='softmax'))

nn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
nn.fit(X_train, y_train_nn, validation_split=0.2, epochs=50, batch_size=16, verbose=0)

loss, acc_nn = nn.evaluate(X_test, y_test_nn, verbose=0)

# =====================
# 7. Results
# =====================
results = {
    "Logistic Regression": acc_log,
    "Random Forest": acc_rf,
    "SVM": acc_svm,
    "XGBoost": acc_xgb,
    "Neural Network": acc_nn
}

print("\nModel Accuracies:")
for model, acc in results.items():
    print(f"{model}: {acc:.4f}")


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [08:33:43] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Model Accuracies:
Logistic Regression: 0.8618
Random Forest: 0.8684
SVM: 0.8553
XGBoost: 0.8947
Neural Network: 0.8816


In [18]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Load dataset

df = pd.read_csv("parkinsons_data/pd_speech_features.csv", header=[1])


feature_groups = {
    "Baseline": list(range(2, 22)),       # example: cols 2–21
    "Intensity": list(range(23, 25)),     # cols 22–24
    "Formants": list(range(26, 29)),      # cols 25–28
    "Bandwidth": list(range(30, 33)),     # cols 29–31
    "VocalFold": list(range(34, 55)),     # cols 32–35
    "MFCC": list(range(56, 139)),          # cols 36–48
    "Wavelet": list(range(140, df.shape[1]-1))  # all until the last column before class
}


# New dataframe to hold compressed features
df_grouped = pd.DataFrame(index=df.index)

# Function: compress group with PCA (1 component)
def group_with_pca(df, cols, new_name):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df.iloc[:, cols])
    pca = PCA(n_components=1)
    df_grouped[new_name] = pca.fit_transform(X_scaled).ravel()  # ensure it's 1D

# Apply PCA for each group
for feature, cols in feature_groups.items():
    group_with_pca(df, cols, feature)

# Add class column back
df_grouped["class"] = df["class"].values

# Show result
print(df_grouped.head())
print("Final shape:", df_grouped.shape)



   Baseline  Intensity  Formants  Bandwidth  VocalFold      MFCC    Wavelet  \
0 -0.085724  -0.071582 -1.426155   0.236350  -1.244790 -0.393715 -10.767723   
1  0.227617  -0.611652 -1.421387   0.302269  -1.002787 -0.047907 -11.267947   
2 -1.332900  -1.146479 -1.458608   0.081536  -2.396363  3.380602 -13.057345   
3 -1.933424   0.971632  1.239160   0.117355  -2.039840 -1.687959  -8.277877   
4 -2.210948   0.991717  1.467792  -0.500374  -2.201670 -3.312962  -5.882761   

   class  
0      1  
1      1  
2      1  
3      1  
4      1  
Final shape: (756, 8)


PCA 7 Feature

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score

# ML models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

# =====================
# 1. Load dataset
# =====================

# Features and target
X = df.drop(columns=["class"]).values
y = df["class"].values

# Encode target
le = LabelEncoder()
y = le.fit_transform(y)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Standardize features (important for SVM, Logistic, NN)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# =====================
# 2. Logistic Regression
# =====================
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)
acc_log = accuracy_score(y_test, y_pred)

# =====================
# 3. Random Forest
# =====================
rf = RandomForestClassifier(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
acc_rf = accuracy_score(y_test, y_pred)

# =====================
# 4. Support Vector Machine
# =====================
svm = SVC(kernel='rbf', probability=True, random_state=42)
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)
acc_svm = accuracy_score(y_test, y_pred)

# =====================
# 5. XGBoost
# =====================
xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
acc_xgb = accuracy_score(y_test, y_pred)

# =====================
# 6. Neural Network (Deep Learning)
# =====================
# One-hot encode target
y_train_nn = to_categorical(y_train)
y_test_nn = to_categorical(y_test)

nn = Sequential()
nn.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
nn.add(Dropout(0.3))
nn.add(Dense(32, activation='relu'))
nn.add(Dense(y_train_nn.shape[1], activation='softmax'))

nn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
nn.fit(X_train, y_train_nn, validation_split=0.2, epochs=50, batch_size=16, verbose=0)

loss, acc_nn = nn.evaluate(X_test, y_test_nn, verbose=0)

# =====================
# 7. Results
# =====================
results = {
    "Logistic Regression": acc_log,
    "Random Forest": acc_rf,
    "SVM": acc_svm,
    "XGBoost": acc_xgb,
    "Neural Network": acc_nn
}

print("\nModel Accuracies:")
for model, acc in results.items():
    print(f"{model}: {acc:.4f}")


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [08:34:04] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Model Accuracies:
Logistic Regression: 0.8618
Random Forest: 0.8684
SVM: 0.8553
XGBoost: 0.8947
Neural Network: 0.8947


**PCA for each Feature**
